In [1]:
import underthesea
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
import joblib


In [2]:
def lower_tokenize(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    tokens = underthesea.word_tokenize(text)
    # print("Lower",tokens)
    return tokens

In [3]:
def remove_stopword(tokens):
    with open("vietnamese-stopwords.txt","r",encoding="utf-8") as file:
        stopwords = file.readlines()
    stopwords = [word.strip() for word in stopwords]
    final_tokens = [token for token in tokens if token not in stopwords]
    final_text = ' '.join(final_tokens)
    # print("Stopword",final_text)
    return final_text


In [4]:
df = pd.read_csv('../Crawl/Data.csv')

In [5]:
# df = df.head(1000)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19805 entries, 0 to 19804
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Danh mục  19805 non-null  object
 1   Nội dung  19805 non-null  object
dtypes: object(2)
memory usage: 309.6+ KB


In [7]:
df['Danh mục'].value_counts()

Danh mục
Văn hóa     2823
Xe          2466
Giới trẻ    2460
Thể thao    2041
Sức khỏe    1906
Giáo dục    1862
Thế giới    1732
Thời sự     1520
Giải trí    1002
Đời sống     979
Kinh tế      819
Du lịch      195
Name: count, dtype: int64

In [8]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19805 entries, 0 to 19804
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Danh mục  19805 non-null  object
 1   Nội dung  19805 non-null  object
dtypes: object(2)
memory usage: 309.6+ KB


In [9]:
content=df['Nội dung'].apply(lambda x: lower_tokenize(x))
content=content.apply(lambda x: remove_stopword(x))
label=df['Danh mục']

In [10]:
content

0        nttt ngụ phú nhuận tphcm bệnh nhân bệnh thoát ...
1        sóng gió đời tư ngọc trinh trở lại vai diễn út...
2        phó tổng thống philippines sara duterteảnh reu...
3        dự thảo quy chế tổ chức thi chứng chỉ ngoại ng...
4        kem trị sẹo thâm chân mua kem trị sẹo thâm châ...
                               ...                        
19800    những tưởng âm thầm biến danh mục sản phẩm yam...
19801    sụt giảm ô tô nhập khẩu việt nam đà tăng trưởn...
19802                                                     
19803    nhu cầu đa dạng khách hàng việt nam doanh nghi...
19804    đọc viết phân biệt đối xử khách hàng việt nam ...
Name: Nội dung, Length: 19805, dtype: object

In [11]:
label

0        Sức khỏe
1        Giải trí
2        Thế giới
3        Giáo dục
4        Sức khỏe
           ...   
19800          Xe
19801          Xe
19802          Xe
19803          Xe
19804          Xe
Name: Danh mục, Length: 19805, dtype: object

In [12]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(content, label, test_size=0.3, random_state=10)
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
# Mã hóa nhãn mục tiêu
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [13]:
X_train_tfidf.shape

(13863, 5000)

In [14]:
X_test_tfidf.shape

(5942, 5000)

In [15]:
#Huấn luyện mô hình Naive Bayes
MultiNB = naive_bayes.MultinomialNB(alpha=0.2)
# fit the training dataset on the classifier
MultiNB.fit(X_train_tfidf, y_train)
# Dự đoán nhãn
predictions_nb = MultiNB.predict(X_test_tfidf)

In [16]:
# Accuracy
accuracy = accuracy_score(predictions_nb, y_test)
print(f"Đánh giá bằng: Accuracy: {accuracy}")

# Precision
precision = precision_score(predictions_nb, y_test, average='weighted')
print(f"Đánh giá bằng: Precision: {precision}")

# Recall
recall = recall_score(predictions_nb, y_test, average='weighted')
print(f"Đánh giá bằng: Recall: {recall}")

# F1 Score
f1 = f1_score(predictions_nb, y_test, average='weighted')
print(f"Đánh giá bằng: F1 Score: {f1}")

Đánh giá bằng: Accuracy: 0.8232918209357118
Đánh giá bằng: Precision: 0.8222117005270214
Đánh giá bằng: Recall: 0.8232918209357118
Đánh giá bằng: F1 Score: 0.8213610897950248


In [17]:
# Chuẩn bị dữ liệu kiểm thử mới
new_data = ["Tôi thích học Toán",
            "VIệt Nam ra sân thắng Lào với tỉ số 10-0",
            "Bạo lực học đường là vấn đề được quan tâm nhất hiện nay",
            "Xe Ariblade 165CC có ngoại hình siêu đẹp"]

# Tiền xử lý dữ liệu kiểm thử mới
new_data= [lower_tokenize(text) for text in new_data]
new_data = [remove_stopword(tokens) for tokens in new_data] # Loại bỏ stop words
# Chuyển đổi dữ liệu kiểm thử mới thành đặc trưng TF-IDF
new_data_tfidf = tfidf_vectorizer.transform(new_data)

# Dự đoán nhãn với dữ liệu kiểm thử mới
new_predictions = MultiNB.predict(new_data_tfidf)

# Chuyển đổi nhãn được dự đoán về dạng ban đầu
new_predictions_labels = encoder.inverse_transform(new_predictions)
# In ra các dự đoán
for i, text in enumerate(new_data):
    print(f"Nội dung: ",text)
    print(f"Dự đoán danh mục: ",new_predictions_labels[i])
    print("-" * 40)

Nội dung:  học toán
Dự đoán danh mục:  Giáo dục
----------------------------------------
Nội dung:  việt nam sân thắng lào tỉ số
Dự đoán danh mục:  Thể thao
----------------------------------------
Nội dung:  bạo lực học đường
Dự đoán danh mục:  Giáo dục
----------------------------------------
Nội dung:  xe ariblade cc ngoại hình siêu đẹp
Dự đoán danh mục:  Xe
----------------------------------------


In [18]:
joblib.dump((tfidf_vectorizer,MultiNB,encoder), "model.joblib")

['model.joblib']